In [20]:
import pandas as pd
import numpy as np
import glob as glob
import matplotlib.pyplot as plt
import scipy.signal as signal
import pickle
import os

In [43]:
def MSCohere_plot(coher_subset, f):
    plt.plot(coher_subset['frequency'], coher_subset['nectary_body'], 'o--k', label = 'nectary-body')
    plt.plot(coher_subset['frequency'], coher_subset['nectary_pilifer'], 'o--r', label = 'nectary-pilifer')
    plt.plot(coher_subset['frequency'], coher_subset['pilifer_body'], 'o--g', label = 'pilifer-body')
    plt.legend()
    plt.ylabel ('Magnitude squared coherence')
    plt.xlabel (' Frequency (Hz)')

In [25]:
file_folder = './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence/'
coherence_files = glob.glob(file_folder + '*MV*')

fft_freq_folder = './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT/'
fft_files = glob.glob(fft_freq_folder + '*MV*Welch*')

In [26]:
coherence_files

['./dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m11_130802.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m13_130802.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m14_130806.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m1_130802.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m4_130802.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m5_130802.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m8_130805.pkl',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence\\Coherence_MV_m9_130801.pkl']

In [27]:
fft_files

['./dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m11_130802values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m13_130802values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m14_130806values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m1_130802values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m4_130802values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m5_130802values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m8_130805values_at_peak_usingWelch.csv',
 './dataFolder/TrackingData_2022_11_10/ErrorAngle/FFT\\MV_m9_130801values_at_peak_usingWelch.csv']

In [48]:
# get coherence at revelant frequencies
f = plt.figure()
for file in coherence_files:
    
    try:
#     # Open the file in read-binary mode
#         with open(file, 'rb') as ff:
#             # Load the object from the file
#             data = pickle.load(ff)

            coherence = pd.DataFrame(pd.read_pickle(file))
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        
    mothID = '_'.join(file.split('\\')[-1][:-4].split('_')[1:])
    
    fft = pd.read_csv([ff for ff in fft_files if mothID in ff][0], index_col=0)
    
    coher_subset = coherence[coherence['frequency'].isin(fft['freqs'])]
    MSCohere_plot(coher_subset, f)
    f.savefig('./dataFolder/TrackingData_2022_11_10/ErrorAngle/Plots/MScohere/' + 'compare_' + mothID + '.png')
    f.clf()

<Figure size 640x480 with 0 Axes>

In [ ]:
fig_directory_name = './dataFolder/TrackingData_2022_11_10/ErrorAngle/Plots/MSCohere/'
output_directory = './dataFolder/TrackingData_2022_11_10/ErrorAngle/MSCoherence/'
fig = plt.figure()
freq = 100
# dt = 1/freq

for f in pilifer_signal_files:
    pilifer_signal = pd.read_csv(f, index_col=0)
    mothID = f.split('\\')[-1][:-15]
    
    trackPoints = pd.read_csv([file for file in trackPoints_files if mothID in file][0], index_col=0)
    
    pilifer_signal_smooth = signal.savgol_filter(pilifer_signal.values.flatten(), 5, 3)
    pilifer_signal_smooth_deg = pilifer_signal_smooth*(180/np.pi)
    
    body_lateral = trackPoints.loc[:,'body_x']
    nectary_lateral = trackPoints.loc[:,'nectary_x']
    
    freq,nectary_pilifer_Cxy = cohere(nectary_lateral, pilifer_signal_smooth_deg, fs = freq)
    freq_1,pilifer_body_Cxy = cohere(pilifer_signal_smooth_deg, body_lateral, fs = freq)
    freq_2, nectary_body_Cxy = cohere(nectary_lateral, body_lateral, fs = freq)
    

    # Store in a dictionary (recommended for easier retrieval)
    data_dict = {
        'frequency': freq,
        'nectary_pilifer': nectary_pilifer_Cxy,
        'pilifer_body': pilifer_body_Cxy,
        'nectary_body': nectary_body_Cxy
    }

    # Define the filename for the pickle file
    filename = 'Coherence_' + mothID + '.pkl'
    full_path = os.path.join(output_directory, filename)

    # Dump the data to the pickle file
    # Using the dictionary for demonstration
    with open(full_path, 'wb') as pickle_f:
        pickle.dump(data_dict, pickle_f)
    
    
    plt.plot(freq, nectary_pilifer_Cxy, label = 'nectary_pilifer')
    plt.plot(freq, pilifer_body_Cxy, label = 'pilifer_body')
    plt.plot(freq, nectary_body_Cxy, label = 'nectary_body')
    plt.legend()
    fig.savefig(fig_directory_name + mothID + '_MSCohere.png')
    plt.cla()